# Constructed Attributes and Model Defaults

In [1]:
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType

In the other introductory notebooks, there were repeated references to default parameters and constructed inputs. What's up with that? Read on for the answers!

## Just Gimme Whatever: Default AgentType Options

Each `AgentType` subclass, like `PerfForesightConsumerType` and `IndShockConsumerType`, has a dictionary called `default_` as a class attribute. It (usually) has three entries: `params`, `solver`, and `model`. The `solver` entry has a reference to the function that solves the one-period problem for the model (conditional on the solution to the succeeding period and the parameters for that period). The `model` entry is a reference to a filename that is used for the new, developmental simulation structure. Finally, the `params` entry has a reference to a dictionary of default model parameters, which is usually defined *just above* the class definition.

### Changing Parameter Values

Whenever you make a new instance of an `AgentType` subclass, you can pass as many arguments as you would like (in a dictionary or otherwise), but they must all be keyword arguments (e.g. `MyType = IndShockConsumerType('CRRA' = 3.0)`, but not `MyType = IndShockConsumerType(3.0)`. Arguments that are explicitly passed will supercede any default value for that parameter. For any parameters that you *don't* pass, the value in the default dictionary will be used instead.

For most `AgentType` subclasses, the default parameters include `T_cycle=1` and `cycles=1`. Thus if you make an instance of that class using only default parameters (no arguments passed), it will represent an agent that lives for only one non-trivial period. That's not very interesting, but it *is* a model! These simple defaults were chosen so that it's fast and easy to make *some* instance of each class, if only so that the user can investigate what's inside it.

The primary exception to the convention that default parameters have `cycles=1` are models for which there is no meaningful interpretation to a finite horizon model. This includes the `TractableBufferStockConsumerType`, which is by defintion infinite horizon (and uses a much different solution method than other HARK models) and the `RepresentativeAgentConsumerType`.

If you are exploring an `AgentType` subclass and want it to do something more interesting, the simplest approach is to set `cycles=0` and then `solve()` your instance. This will solve the *infinite horizon* model with default parameters. Alternatively, set `cycles>1` to solve several non-terminal (or non-trivial) periods.

### Style Conventions for Default Parameters

To the greatest extent possible, we have tried to make all default parameters be single numbers, or a list with a single number. For anything more complex than that, there should be a **constructor** function (see below) to build the more complicated object.

In some cases, it would be awkward or impossible to specify simple parameters, and a larger object needs to be passed as arguments to an `AgentType` subclass's initializer. That's fine, and we do it too in our projects! The most common situation for this is when some model inputs have been pre-computed-- maybe in an earlier phase of this project, or maybe the results from someone else's project. This might be a life-cycle sequence of parameters estimated in reduced form, or transition probabilities among discrete states. In these situations, we recommend that your project have a `Parameters.py` file where you do the work of reading in this outside data and constructing the parameter dictionary you want to use.

## Constructor Functions for Complex Model Inputs

Some objects needed to solve a model are complex, but can be created based on simpler "primitive" parameters. This includes the lifecycle sequence of income shock distributions, a parameterizable Markov transition matrix among discrete states, or age-dependent mortality probabilities that can be characterized with polynomial coefficients.

HARK can automatically build these objects using constructor functions. Such functions are specified in each instance's `constructors` dictionary, which can be passed like any other object at initialization. Each `AgentType` has default constructors for some model objects, which are listed in the `constructors` entry of its default parameter dictionary.

### Viewing Constructors

The easiest way to 